# Функции и import-и на библиотеки

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, MaxPooling1D, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras import Input
from tensorflow.keras import layers, optimizers, regularizers
from matplotlib import pyplot as plt

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
def one_hot_encoder(f):
    if f < 6:
        return [1,0,0]
    if f == 6:
        return [0,1,0]
    else:
        return [0,0,1]

In [4]:
def bin_and_replace(feature_name, num_bins):
  dataset[feature_name]=pd.qcut(dataset[feature_name], q=num_bins)
  dataset[feature_name] = dataset[feature_name].map(lambda x: float(x.mid))

# Вчитување на податоците

In [5]:
dataset = pd.read_csv('winequality-white.csv', delimiter=';')

In [6]:
dataset.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


# Предпроцесирање на податоците

## One-hot енкодирање 


In [7]:
dataset['quality'] = dataset['quality'].map(lambda x: one_hot_encoder(x))

После овој чекор, класната лабела нема да биде цел број, туку ќе биде вектор добиен со помош на one-hot енкодирањето.

In [8]:
dataset.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,"[0, 1, 0]"
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,"[0, 1, 0]"
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,"[0, 1, 0]"
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,"[0, 1, 0]"
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,"[0, 1, 0]"


## Нормализација на карактеристиките

Правиме нормализација на карактеристиките, така што сите ќе имаат вредности од 0 до 1.

In [9]:
scaler = MinMaxScaler()
dataset[dataset.columns[0:-1]] = scaler.fit_transform(dataset[dataset.columns[0:-1]].values)

За проверка на успешноста на нормализацијата, може да забележеме дека max вредност за секоја карактеристика изнесува 1.

In [10]:
pd.DataFrame(dataset.describe().iloc[-1])

,max
fixed acidity,1.0
volatile acidity,1.0
citric acid,1.0
residual sugar,1.0
chlorides,1.0
free sulfur dioxide,1.0
total sulfur dioxide,1.0
density,1.0
pH,1.0
sulphates,1.0


Ако го прикажеме податочното множество може да забележиме дека истото веќе е изменето.

In [11]:
dataset.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0.307692,0.186275,0.216867,0.308282,0.106825,0.149826,0.373550,0.267785,0.254545,0.267442,0.129032,"[0, 1, 0]"
1,0.240385,0.215686,0.204819,0.015337,0.118694,0.041812,0.285383,0.132832,0.527273,0.313953,0.241935,"[0, 1, 0]"
2,0.413462,0.196078,0.240964,0.096626,0.121662,0.097561,0.204176,0.154039,0.490909,0.255814,0.338710,"[0, 1, 0]"
3,0.326923,0.147059,0.192771,0.121166,0.145401,0.156794,0.410673,0.163678,0.427273,0.209302,0.306452,"[0, 1, 0]"
4,0.326923,0.147059,0.192771,0.121166,0.145401,0.156794,0.410673,0.163678,0.427273,0.209302,0.306452,"[0, 1, 0]"


## Binning

Следно, за секоја карактеристика пооделно да го увидиме нејзиниот домен, поточно неговата кардиналност.

In [12]:
col_1 = []
col_2 = []
for feature in dataset.columns[:-1]:
  col_1.append(feature)
  col_2.append(len(dataset[feature].value_counts()))

domain_cardinalities = pd.DataFrame ({'Feature':col_1, 'Cardinality of domain':col_2}, columns = ['Feature','Cardinality of domain'])

In [13]:
domain_cardinalities

,Feature,Cardinality of domain
0,fixed acidity,68
1,volatile acidity,125
2,citric acid,87
3,residual sugar,310
4,chlorides,160
5,free sulfur dioxide,132
6,total sulfur dioxide,251
7,density,890
8,pH,103
9,sulphates,79


Може да се забележи дека одрдени карактеристики како например density имаат големи домени, тие големи домени треба некако да ги дискретизираме, со што ќе станат помали, па ќе направиме некакво стабилизарање на податоците.

Како резултат од binning-от, секоја конкретна вредност за сите карактеристики, кои досега се реални броеви, ќе стане интервал (податочна структура од pandas), па секој интервал ќе го замениме со неговата мода (вредноста на средината).

In [14]:
# for feature in dataset.columns[:-1]:
#   num_ftrs = len(dataset[feature].value_counts())
#   if num_ftrs < 100:
#     bin_and_replace(feature, 5)
#   elif 100 <= num_ftrs < 300:
#     bin_and_replace(feature, 8)
#   else:
#     bin_and_replace(feature, 10)

In [15]:
col_1 = []
col_2 = []
for feature in dataset.columns[:-1]:
  col_1.append(feature)
  col_2.append(len(dataset[feature].value_counts()))

domain_cardinalities = pd.DataFrame ({'Feature':col_1, 'Cardinality of domain':col_2}, columns = ['Feature','Cardinality of domain'])

In [16]:
domain_cardinalities

,Feature,Cardinality of domain
0,fixed acidity,68
1,volatile acidity,125
2,citric acid,87
3,residual sugar,310
4,chlorides,160
5,free sulfur dioxide,132
6,total sulfur dioxide,251
7,density,890
8,pH,103
9,sulphates,79


In [17]:
dataset.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0.307692,0.186275,0.216867,0.308282,0.106825,0.149826,0.373550,0.267785,0.254545,0.267442,0.129032,"[0, 1, 0]"
1,0.240385,0.215686,0.204819,0.015337,0.118694,0.041812,0.285383,0.132832,0.527273,0.313953,0.241935,"[0, 1, 0]"
2,0.413462,0.196078,0.240964,0.096626,0.121662,0.097561,0.204176,0.154039,0.490909,0.255814,0.338710,"[0, 1, 0]"
3,0.326923,0.147059,0.192771,0.121166,0.145401,0.156794,0.410673,0.163678,0.427273,0.209302,0.306452,"[0, 1, 0]"
4,0.326923,0.147059,0.192771,0.121166,0.145401,0.156794,0.410673,0.163678,0.427273,0.209302,0.306452,"[0, 1, 0]"


# Поделба на соодветни подмножества

Поделба на тренинг множество (првите 80% од целокупното податочно множество) и тест множество (останатите 20%)

In [18]:
train = dataset[:int(len(dataset)*0.8)]
test = dataset[int(len(dataset)*0.8):]

In [19]:
print("Train set:\t{}".format(train.shape))
print("Test set:\t{}".format(test.shape))

Train set:	(3918, 12)
Test set:	(980, 12)


Поделба на тренинг и тест множествата на X(карактеристики) и Y(класни лабели) подмножества

In [20]:
def split_subset(subset):
  return (subset[subset.columns[0:-1]], subset[subset.columns[-1:]])

In [21]:
train_x, train_y = split_subset(train)
test_x, test_y = split_subset(test)

In [25]:
train_x = np.array(train_x.values.tolist())
train_y = np.array([t[0] for t in train_y.values.tolist()])

test_x = np.array(test_x.values.tolist())
test_y = np.array([t[0] for t in test_y.values.tolist()])

# Тренирање модел на невронска мрежа

In [26]:
# model = Sequential()
# model.add(Input(11))
# model.add(Dense(4, activation='tanh', input_shape=(11,)))
# model.add(Dense(4, activation='tanh', input_shape=(11,)))
# model.add(Dense(3, activation='softmax'))

model = Sequential()
# layer 1
model.add(Dense(30, input_dim=11, activation='relu', name='fc0',kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization(momentum=0.99, epsilon=0.001))
#layer 2
model.add(Dense(50, name='fc1',bias_initializer='zeros'))
model.add(BatchNormalization(momentum=0.99, epsilon=0.001))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
#layer 3
model.add(Dense(100, name='fc2',bias_initializer='zeros'))
model.add(BatchNormalization(momentum=0.99, epsilon=0.001))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#layer 4
model.add(Dense(3, name='fc3',bias_initializer='zeros'))
model.add(BatchNormalization(momentum=0.99, epsilon=0.001))
model.add(Activation('softmax'))

In [27]:
BATCH_SIZE=128
EPOCHS=500
LEARNING_RATE = 0.05

optimizer = Adam(lr=LEARNING_RATE)
model.compile(optimizer=optimizer, loss=categorical_crossentropy, metrics=['accuracy'])
history = model.fit(x=train_x, y=train_y, validation_split=0.125, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1)

Epoch 1/500
27/27 [==============================] - 1s 20ms/step - loss: 1.0636 - accuracy: 0.5018 - val_loss: 3.2746 - val_accuracy: 0.2714
Epoch 2/500
27/27 [==============================] - 0s 4ms/step - loss: 0.9451 - accuracy: 0.5327 - val_loss: 1.0591 - val_accuracy: 0.4592
Epoch 3/500
27/27 [==============================] - 0s 5ms/step - loss: 0.9370 - accuracy: 0.5286 - val_loss: 0.9591 - val_accuracy: 0.4980
Epoch 4/500
27/27 [==============================] - 0s 6ms/step - loss: 0.9176 - accuracy: 0.5481 - val_loss: 1.0013 - val_accuracy: 0.4510
Epoch 5/500
27/27 [==============================] - 0s 2ms/step - loss: 0.9262 - accuracy: 0.5438 - val_loss: 0.9884 - val_accuracy: 0.4449
Epoch 6/500
27/27 [==============================] - 0s 3ms/step - loss: 0.9296 - accuracy: 0.5370 - val_loss: 0.9568 - val_accuracy: 0.5306
Epoch 7/500
27/27 [==============================] - 0s 5ms/step - loss: 0.9217 - accuracy: 0.5385 - val_loss: 0.9939 - val_accuracy: 0.4490
Epoch 8/500


Epoch 59/500
27/27 [==============================] - 0s 6ms/step - loss: 0.9061 - accuracy: 0.5563 - val_loss: 0.8976 - val_accuracy: 0.5592
Epoch 60/500
27/27 [==============================] - 0s 4ms/step - loss: 0.9055 - accuracy: 0.5481 - val_loss: 0.9441 - val_accuracy: 0.5204
Epoch 61/500
27/27 [==============================] - 0s 5ms/step - loss: 0.9080 - accuracy: 0.5513 - val_loss: 0.9013 - val_accuracy: 0.5755
Epoch 62/500
27/27 [==============================] - 0s 9ms/step - loss: 0.9137 - accuracy: 0.5438 - val_loss: 0.9072 - val_accuracy: 0.5694
Epoch 63/500
27/27 [==============================] - 0s 3ms/step - loss: 0.9094 - accuracy: 0.5595 - val_loss: 0.8981 - val_accuracy: 0.5796
Epoch 64/500
27/27 [==============================] - 0s 4ms/step - loss: 0.9099 - accuracy: 0.5481 - val_loss: 0.9330 - val_accuracy: 0.5510
Epoch 65/500
27/27 [==============================] - 0s 4ms/step - loss: 0.9160 - accuracy: 0.5519 - val_loss: 0.9508 - val_accuracy: 0.5122
Epoch 

27/27 [==============================] - 0s 9ms/step - loss: 0.9092 - accuracy: 0.5499 - val_loss: 0.8787 - val_accuracy: 0.6082
Epoch 117/500
27/27 [==============================] - 0s 9ms/step - loss: 0.9058 - accuracy: 0.5563 - val_loss: 1.0196 - val_accuracy: 0.4939
Epoch 118/500
27/27 [==============================] - 0s 9ms/step - loss: 0.9018 - accuracy: 0.5546 - val_loss: 0.8848 - val_accuracy: 0.5776
Epoch 119/500
27/27 [==============================] - 0s 9ms/step - loss: 0.9148 - accuracy: 0.5455 - val_loss: 0.9344 - val_accuracy: 0.5265
Epoch 120/500
27/27 [==============================] - 0s 9ms/step - loss: 0.9068 - accuracy: 0.5624 - val_loss: 0.8924 - val_accuracy: 0.5980
Epoch 121/500
27/27 [==============================] - 0s 9ms/step - loss: 0.9108 - accuracy: 0.5651 - val_loss: 0.8675 - val_accuracy: 0.5776
Epoch 122/500
27/27 [==============================] - 0s 10ms/step - loss: 0.9036 - accuracy: 0.5598 - val_loss: 0.8929 - val_accuracy: 0.5469
Epoch 123/50

27/27 [==============================] - 0s 10ms/step - loss: 0.9090 - accuracy: 0.5589 - val_loss: 1.2072 - val_accuracy: 0.3796
Epoch 174/500
27/27 [==============================] - 0s 11ms/step - loss: 0.9181 - accuracy: 0.5513 - val_loss: 0.8947 - val_accuracy: 0.5633
Epoch 175/500
27/27 [==============================] - 0s 3ms/step - loss: 0.9065 - accuracy: 0.5578 - val_loss: 0.9969 - val_accuracy: 0.5184
Epoch 176/500
27/27 [==============================] - 0s 3ms/step - loss: 0.9107 - accuracy: 0.5537 - val_loss: 0.9141 - val_accuracy: 0.5796
Epoch 177/500
27/27 [==============================] - 0s 5ms/step - loss: 0.9083 - accuracy: 0.5516 - val_loss: 0.8586 - val_accuracy: 0.5980
Epoch 178/500
27/27 [==============================] - 0s 10ms/step - loss: 0.9153 - accuracy: 0.5475 - val_loss: 0.9477 - val_accuracy: 0.5286
Epoch 179/500
27/27 [==============================] - 0s 9ms/step - loss: 0.9148 - accuracy: 0.5636 - val_loss: 0.8906 - val_accuracy: 0.5878
Epoch 180/

27/27 [==============================] - 0s 9ms/step - loss: 0.9020 - accuracy: 0.5546 - val_loss: 0.9276 - val_accuracy: 0.5408
Epoch 231/500
27/27 [==============================] - 0s 9ms/step - loss: 0.8965 - accuracy: 0.5598 - val_loss: 0.8734 - val_accuracy: 0.6102
Epoch 232/500
27/27 [==============================] - 0s 9ms/step - loss: 0.8910 - accuracy: 0.5694 - val_loss: 0.9269 - val_accuracy: 0.5796
Epoch 233/500
27/27 [==============================] - 0s 10ms/step - loss: 0.9088 - accuracy: 0.5572 - val_loss: 0.9738 - val_accuracy: 0.5265
Epoch 234/500
27/27 [==============================] - 0s 10ms/step - loss: 0.9065 - accuracy: 0.5624 - val_loss: 1.0201 - val_accuracy: 0.5163
Epoch 235/500
27/27 [==============================] - 0s 11ms/step - loss: 0.8937 - accuracy: 0.5557 - val_loss: 1.0995 - val_accuracy: 0.4755
Epoch 236/500
27/27 [==============================] - 0s 10ms/step - loss: 0.9026 - accuracy: 0.5633 - val_loss: 0.9627 - val_accuracy: 0.5286
Epoch 237

27/27 [==============================] - 0s 9ms/step - loss: 0.9001 - accuracy: 0.5583 - val_loss: 0.9070 - val_accuracy: 0.5510
Epoch 288/500
27/27 [==============================] - 0s 9ms/step - loss: 0.9045 - accuracy: 0.5616 - val_loss: 0.9061 - val_accuracy: 0.5551
Epoch 289/500
27/27 [==============================] - 0s 9ms/step - loss: 0.8993 - accuracy: 0.5645 - val_loss: 0.9547 - val_accuracy: 0.4918
Epoch 290/500
27/27 [==============================] - 0s 9ms/step - loss: 0.8879 - accuracy: 0.5598 - val_loss: 0.9377 - val_accuracy: 0.5286
Epoch 291/500
27/27 [==============================] - 0s 9ms/step - loss: 0.8897 - accuracy: 0.5703 - val_loss: 0.9246 - val_accuracy: 0.5245
Epoch 292/500
27/27 [==============================] - 0s 10ms/step - loss: 0.8928 - accuracy: 0.5575 - val_loss: 0.9222 - val_accuracy: 0.5531
Epoch 293/500
27/27 [==============================] - 0s 10ms/step - loss: 0.9004 - accuracy: 0.5546 - val_loss: 0.8772 - val_accuracy: 0.5653
Epoch 294/5

27/27 [==============================] - 0s 2ms/step - loss: 0.8884 - accuracy: 0.5627 - val_loss: 1.0084 - val_accuracy: 0.5020
Epoch 345/500
27/27 [==============================] - 0s 2ms/step - loss: 0.8993 - accuracy: 0.5607 - val_loss: 0.9722 - val_accuracy: 0.5061
Epoch 346/500
27/27 [==============================] - 0s 3ms/step - loss: 0.8958 - accuracy: 0.5630 - val_loss: 0.8743 - val_accuracy: 0.5653
Epoch 347/500
27/27 [==============================] - 0s 9ms/step - loss: 0.9004 - accuracy: 0.5677 - val_loss: 1.0358 - val_accuracy: 0.4918
Epoch 348/500
27/27 [==============================] - 0s 8ms/step - loss: 0.8996 - accuracy: 0.5648 - val_loss: 0.9038 - val_accuracy: 0.5939
Epoch 349/500
27/27 [==============================] - 0s 2ms/step - loss: 0.9009 - accuracy: 0.5607 - val_loss: 0.8847 - val_accuracy: 0.5694
Epoch 350/500
27/27 [==============================] - 0s 3ms/step - loss: 0.8880 - accuracy: 0.5686 - val_loss: 0.8600 - val_accuracy: 0.5796
Epoch 351/500

27/27 [==============================] - 0s 12ms/step - loss: 0.8999 - accuracy: 0.5569 - val_loss: 0.9434 - val_accuracy: 0.5653
Epoch 402/500
27/27 [==============================] - 0s 9ms/step - loss: 0.8955 - accuracy: 0.5642 - val_loss: 0.8887 - val_accuracy: 0.5776
Epoch 403/500
27/27 [==============================] - 0s 8ms/step - loss: 0.9120 - accuracy: 0.5578 - val_loss: 1.0460 - val_accuracy: 0.4898
Epoch 404/500
27/27 [==============================] - 0s 2ms/step - loss: 0.9022 - accuracy: 0.5575 - val_loss: 0.8997 - val_accuracy: 0.5449
Epoch 405/500
27/27 [==============================] - 0s 2ms/step - loss: 0.8902 - accuracy: 0.5706 - val_loss: 0.8622 - val_accuracy: 0.5857
Epoch 406/500
27/27 [==============================] - 0s 6ms/step - loss: 0.9004 - accuracy: 0.5601 - val_loss: 0.8529 - val_accuracy: 0.5490
Epoch 407/500
27/27 [==============================] - 0s 9ms/step - loss: 0.8858 - accuracy: 0.5715 - val_loss: 0.9505 - val_accuracy: 0.5061
Epoch 408/50

27/27 [==============================] - 0s 9ms/step - loss: 0.8976 - accuracy: 0.5557 - val_loss: 0.9586 - val_accuracy: 0.5000
Epoch 459/500
27/27 [==============================] - 0s 9ms/step - loss: 0.8926 - accuracy: 0.5648 - val_loss: 0.9303 - val_accuracy: 0.5327
Epoch 460/500
27/27 [==============================] - 0s 9ms/step - loss: 0.8952 - accuracy: 0.5648 - val_loss: 0.9128 - val_accuracy: 0.5694
Epoch 461/500
27/27 [==============================] - 0s 9ms/step - loss: 0.9014 - accuracy: 0.5586 - val_loss: 1.0308 - val_accuracy: 0.4347
Epoch 462/500
27/27 [==============================] - 0s 9ms/step - loss: 0.8960 - accuracy: 0.5551 - val_loss: 0.9306 - val_accuracy: 0.5327
Epoch 463/500
27/27 [==============================] - 0s 9ms/step - loss: 0.8870 - accuracy: 0.5592 - val_loss: 1.0081 - val_accuracy: 0.4041
Epoch 464/500
27/27 [==============================] - 0s 9ms/step - loss: 0.8982 - accuracy: 0.5659 - val_loss: 0.9496 - val_accuracy: 0.5286
Epoch 465/500

In [ ]:
# Анализа на тренирањето

# Анализа на тренирањето

In [ ]:
# TODO: Visualize loss
def visualize(metric):
    plt.rcParams["figure.figsize"] = (30,10)
    plt.plot(history.history[f'{metric}'])
    plt.plot(history.history[f'val_{metric}'])
    plt.title(f'Model {metric}')
    plt.ylabel(f'{metric}')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [ ]:
visualize("accuracy")

In [ ]:
visualize("loss")

# Валидација на невронската мрежа врз тест множеството

In [ ]:
results_test = model.evaluate(test_x,test_y, batch_size=128)

# Тренирање други класификатори

In [ ]:
def one_hot_decode(encoded):
  for i in range(0, len(encoded)):
    if encoded[i] == 1:
      return i
  return -1

In [ ]:
train_y_decoded = [one_hot_decode(cl) for cl in train_y.tolist()]

In [ ]:
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
clf = SVC()
scoring = 'accuracy'
score = cross_val_score(clf, train_x.tolist(), train_y_decoded, cv=k_fold, n_jobs=1, scoring=scoring)
print("SVM: {}".format(round(np.mean(score)*100, 2)))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

dt_clf = DecisionTreeClassifier(criterion='entropy')

k_fold = KFold(n_splits=10, shuffle=True)
scoring = 'accuracy'
score = cross_val_score(dt_clf, train_x.tolist(), train_y_decoded, cv=k_fold, n_jobs=1, scoring=scoring)
print("Decision Tree: {}".format(round(np.mean(score)*100, 2)))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rt_clf = RandomForestClassifier(n_estimators=10, criterion='gini')

k_fold = KFold(n_splits=10, shuffle=True)
scoring = 'accuracy'
score = cross_val_score(dt_clf, train_x.tolist(), train_y_decoded, cv=k_fold, n_jobs=1, scoring=scoring)
print("Random Forest with (100 trees): {}".format(round(np.mean(score)*100, 2)))

# Сумаризација на резултатите од класификаторите